# Solving the 1D fluid flow problem using PINNs (Physics Informed Neural Networks)

## Problem Statement 

let us have the following problem 

$$
\frac{\partial}{\partial x}\left(\frac{\beta_c k A}{\mu B} \frac{\partial p}{\partial x}\right) \Delta x+q_{s c}=\frac{V_b \phi c_t}{\alpha_c B} \frac{\partial p}{\partial t}
$$

Assume that the viscosity and formation volume factor are constant and using the following data: Length $=2500 \mathrm{ft}$, Width $=80 \mathrm{ft}$, thickness $=60 \mathrm{ft} \quad B=1.2 \mathrm{bbl} / \mathrm{stb}, \alpha_c=5.615, \beta_c=1.127 \times 10^{-3}$ Well radius $r_w=0.33 \mathrm{ft}, \quad$ Viscosity $\mu=1.3 \mathrm{cpt}=1.3 \mathrm{cp}$

Total compressibility $c_t=7.3 \times 10^{-6} \mathrm{psi}^{-1} \quad$ Initial pressure $p_i=5000 \mathrm{psi}$ in all grid blocks Constant injection rate $700 \mathrm{stb} /$ day from Grid \# 2 . Constant production rate of $900 \mathrm{stb} /$ day at Grid \# 4 . There is a leakage of 250bbl/day of fluid at the left boundary $(x=0)$ and the pressure is held constant at the right boundary $(x=L)$ at 5200 psi and time-step size is 0.2 day. The table below gives the properties of each grid block:

### Setting the variables and constants

Let us have 
$$ \lambda = \dfrac{\beta_c A}{\mu B} \quad \& \quad \gamma = \dfrac{V_b c_t}{\alpha_c B}$$
So out problem can be written as 
 $$
\frac{\partial}{\partial x} \left(\lambda k \frac{\partial p}{\partial x}\right) \Delta x+q_{s c}=\gamma \phi \frac{\partial p}{\partial t}
$$


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt

In [10]:
# set the device to be a GPU, if it is there
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### Setting the NEural network 

In [3]:
# the model
class FCN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
            nn.Linear(N_INPUT, N_HIDDEN),
            activation()

        ])


        self.fch =nn.Sequential(*[ nn.Sequential(*[
            nn.Linear(N_HIDDEN, N_HIDDEN),
            activation()

        ]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN,N_OUTPUT)
        def weights_initialization(self):
                """"
                When we define all the modules such as the layers in '__init__()'
                method above, these are all stored in 'self.modules()'.
                We go through each module one by one. This is the entire network,
                basically.
                """
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight,gain=1.0)
                nn.init.constant_(m.bias, 0)
    def forward(self, x,t):
        inputs = torch.cat([x,t],axis=1) # combined two arrays of 1 columns each to one array of 2 columns
        inputs = self.fcs(inputs)
        inputs = self.fch(inputs)
        inputs = self.fce(inputs)
        return inputs

Setting the PDE loss

In [23]:
x_0 = 0.0
x_f = 10000.0
h =  100.0

t_0 = 0.0
t_f = 50.0 
k = 0.2 

def generate_data(x_num = int((x_f - x_0)/h), t_num=int((t_f-t_0)/k)):
    x_axis = torch.linspace(x_0, x_f,  x_num + 2)[1:-1] #exclude the boundary
    t_axis = torch.linspace(t_0, t_f, t_num +1)[1:]
    data = torch.cartesian_prod(x_axis, t_axis)
    return torch.split(data, 1, 1)

In [36]:
# Input points from the interior of the domain
x_physics , t_physics = generate_data()
x_physics = x_physics.requires_grad_().to(device)
t_physics = t_physics.requires_grad_().to(device)
examples_num = len(x_physics)

n_of_points_per_grid = examples_num / 100

In [41]:
x_left = torch.tensor( [x_0]*examples_num ).unsqueeze(-1).requires_grad_().to(device)
x_right = torch.tensor( [x_f]*examples_num ).unsqueeze(-1).requires_grad_().to(device)
# Use NumPy to load data from the text file
numpy_perm = np.loadtxt('perm.csv').repeat(n_of_points_per_grid).reshape(-1,1)
numpy_porosity = np.loadtxt('porosity.csv').repeat(n_of_points_per_grid).reshape(-1,1)
# Convert NumPy array to PyTorch tensor
perm = torch.tensor(numpy_perm).requires_grad_()
porosity = torch.tensor(numpy_porosity)
# Now you can use tensor_data in your PyTorch code

t_init = torch.tensor([t_0] *examples_num ).unsqueeze(-1).requires_grad_().to(device)

In [22]:
pinn = FCN(2,1,32,3)
mse_cost_function = torch.nn.MSELoss()
optimizer = optim.Adam(pinn.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.45)

In [ ]:
def pde():
    

In [47]:
total_losses = []
iteration = []
boundary_loss = []
epochs = 2

for epoch in range(1, epochs+1):
    optimizer.zero_grad()

    #initial loss
    p = pinn(x_physics,t_init)
    p_x = torch.autograd.grad(p,x_physics,torch.ones_like(p), create_graph=True)[0]
    p_t = torch.autograd.grad(p,t_init,torch.ones_like(p), create_graph=True)[0]
    ps_xx = torch.autograd.grad(p_x * perm, x_physics, torch.ones_like(p_x), create_graph=True)[0]
    pde_loss = ps_xx * 100 - porosity * p_t 
    mse_cost_function(pde_loss, torch.zeros_like(pde_loss))

    # Boundary loss 
    p = pinn(x_left,t_physics)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [46]:
p.shape

torch.Size([25000, 1])